In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [2]:
%cd /content/drive/MyDrive/enz-eff-project
!pip install -r requirements.txt
%cd improved_code/model_training

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 37.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 55.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.8/144.8 kB 17.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.9/91.9 kB 13.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 8.9 MB/s eta 0:00:00
INFO: pip is looking at multiple versions of torch to determine which version is compatible with other requirements. This could take a while.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.1 MB/s eta 0:00:0

In [3]:
!pip show pandas

Name: pandas
Version: 1.5.3
Summary: Powerful data structures for data analysis, time series, and statistics
Home-page: https://pandas.pydata.org
Author: The Pandas Development Team
Author-email: pandas-dev@python.org
License: BSD-3-Clause
Location: /usr/local/lib/python3.10/dist-packages
Requires: numpy, python-dateutil, pytz
Required-by: altair, arviz, bigframes, bio, bioservices, bokeh, bqplot, cmdstanpy, cobra, cufflinks, datascience, db-dtypes, dopamine-rl, fastai, geemap, geopandas, google-colab, gspread-dataframe, holoviews, ibis-framework, mizani, mlxtend, pandas-datareader, pandas-gbq, panel, plotnine, prophet, pymc, seaborn, sklearn-pandas, statsmodels, vega-datasets, xarray, yfinance


In [1]:
%cd /content/drive/MyDrive/enz-eff-project/improved_code/model_training
import numpy as np
import pickle
import pandas as pd
from os.path import join
import warnings
warnings.filterwarnings("ignore")
from sklearn.metrics import r2_score, mean_squared_error
from scipy import stats
import xgboost as xgb

/content/drive/.shortcut-targets-by-id/1iS6gSWfUE3cZnmrNWbbV9_W_zQH3vaiu/enz-eff-project/improved_code/model_training


## Loading training and test data:

In [2]:
train_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_train_indices.npy"), allow_pickle = True))
test_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_test_indices.npy"), allow_pickle = True))

In [3]:
data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_new_feats.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_new_feats.pkl"))

data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
len(data_train), len(data_test)

(3391, 874)

In [4]:
data_train.columns

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       'socn_normalized', 'qso_combined', 'qso_normalized', 'traid_combined',
       'traid_normalized', 'paac_combined', 'paac_normalized', 'ESM1b_norm',
       'ESM1b_ts_norm', 'log10_kcat_norm'],
      dtype='object', length=2302)

**Saving Xgboost Models**

In [5]:
%ls ../../models/train_models

 difference_fp.h5                          esm1bts_drfp_moreau_broto_auto_combined.h5
 drfp.h5                                   esm1bts_drfp_paac_combined.h5
 esm1b.h5                                  esm1bts_drfp_traid_combined.h5
 esm1bts_all_new_feats.h5                  esm1bts_esm1b_combined_all_feats.h5
 esm1bts_diff.h5                           esm1bts.h5
'esm1bts_drfp (1).h5'                      structural_fp.h5
 esm1bts_drfp_aa_composition.h5            xgboost_diff.h5
 esm1bts_drfp_chem.h5                      xgboost_drfp.h5
 esm1bts_drfp_ctd_combined.h5              xgboost_esm1b.h5
 esm1bts_drfp_dp_composition_combined.h5   xgboost_esm1b_ts_diff.h5
 esm1bts_drfp_geary_auto_combined.h5       xgboost_esm1bts_drfp.h5
 esm1bts_drfp.h5                           xgboost_esm1b_ts.h5
 esm1bts_drfp_maccs.h5                     xgboost_structFp.h5
 esm1bts_drfp_moran_auto_combined.h5       xg_esm1b_esm1bts.h5


In [6]:
def save_pickel_model(model, model_name):
    xgboost_trained_models = "../../models/train_models"
    model_path = join(xgboost_trained_models, model_name)
    with open(model_path, 'wb') as model_file:
        pickle.dump(model, model_file)

## 1. Training a model with only sequence information (ESM-1b):

#### (a) Creating input matrices:

In [7]:
train_ESM1b = np.array(list(data_train["ESM1b"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [8]:
# def cross_validation_mse_gradient_boosting(param):
#     num_round = param["num_rounds"]
#     del param["num_rounds"]
#     param["max_depth"] = int(np.round(param["max_depth"]))
#     param["tree_method"] = "gpu_hist"
#     param["sampling_method"] = "gradient_based"

#     MSE = []
#     R2 = []
#     for i in range(5):
#         train_index, test_index  = train_indices[i], test_indices[i]
#         dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
#         dvalid = xgb.DMatrix(train_X[test_index])
#         bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
#         y_valid_pred = bst.predict(dvalid)
#         MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
#         R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
#     return(-np.mean(R2))


# from hyperopt import fmin, tpe, rand, hp, Trials

# space_gradient_boosting = {
#     "learning_rate": hp.uniform("learning_rate", 0.01, 1),
#     "max_depth": hp.uniform("max_depth", 4,12),
#     #"subsample": hp.uniform("subsample", 0.7, 1),
#     "reg_lambda": hp.uniform("reg_lambda", 0, 5),
#     "reg_alpha": hp.uniform("reg_alpha", 0, 5),
#     "max_delta_step": hp.uniform("max_delta_step", 0, 5),
#     "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
#     "num_rounds":  hp.uniform("num_rounds", 20, 200)}


# trials = Trials()
# best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
#             algo=rand.suggest, max_evals = 200, trials=trials);

  0%|          | 1/200 [00:23<1:17:18, 23.31s/trial, best loss: -0.25992605455628026]


KeyboardInterrupt: 

#### (c) Training and validating model:

In [9]:
param = {'learning_rate': 0.051447544749765035,
         'max_delta_step': 2.956459783615207,
         'max_depth': 5.034202474908222,
         'min_child_weight': 7.457989829577018,
         'num_rounds': 400.50601395689256,
         'reg_alpha': 1.0858835704466614,
         'reg_lambda': 1.1385559144302175}





num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [10]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b.npy"), np.array(R2))'''


'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b.npy"), np.array(R2))'

In [11]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
# MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
MSE_dif_fp_test = mean_squared_error(np.reshape(test_Y, (-1)), y_test_pred)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b.npy"), test_Y)


0.631 0.805 0.398


### Pearson 0.621 MSE 0.823 R2 0.384


In [ ]:
save_pickel_model(bst, "xgboost_esm1b.h5")

In [ ]:
y_test_pred_esm1b = y_test_pred

## 2. Training a model with only sequence information (ESM-1b_ts):

#### (a) Creating input matrices:

In [12]:
train_ESM1b = np.array(list(data_train["ESM1b_ts"]))
train_X = train_ESM1b
train_Y = np.array(list(data_train["log10_kcat"]))

test_ESM1b = np.array(list(data_test["ESM1b_ts"]))
test_X = test_ESM1b
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [13]:

'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [14]:
param = {'learning_rate': 0.2831145406836757,
         'max_delta_step': 0.07686715986169101,
         'max_depth': 4.96836783761305,
          'min_child_weight': 6.905400087083855,
           'num_rounds': 313.1498988074061,
            'reg_alpha': 1.717314107718892,
             'reg_lambda': 2.470354543039016}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [15]:
'''
R2 = []
MSE = []
Pearson = []
y_valid_pred_esm1b_ts = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    y_valid_pred_esm1b_ts.append(y_valid_pred)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b_ts.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\ny_valid_pred_esm1b_ts = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    y_valid_pred_esm1b_ts.append(y_valid_pred)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_ESM1b_ts.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_resul

In [16]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts.npy"), test_Y)

0.649 0.777 0.419


In [17]:
save_pickel_model(bst, "xgboost_esm1b_ts.h5")

In [18]:
y_test_pred_esm1b_ts = y_test_pred

## 3. Training a model with only reaction information (DRFP):

In [19]:
train_X = np.array(list(data_train["DRFP"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["DRFP"]))
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [20]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [21]:
param = {'learning_rate': 0.08987247189322463,
         'max_delta_step': 1.1939737318908727,
         'max_depth': 11.268531225242574,
         'min_child_weight': 2.8172720953826302,
         'num_rounds': 400.03643430746544,
         'reg_alpha': 1.9412226989868904,
         'reg_lambda': 4.950543905603358}


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [22]:
'''
R2 = []
MSE = []
Pearson = []
y_valid_pred_DRFP = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    y_valid_pred_DRFP.append(y_valid_pred)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_DRFP.npy"), np.array(Pearson))
np.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_DRFP.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_DRFP.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\ny_valid_pred_DRFP = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    y_valid_pred_DRFP.append(y_valid_pred)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_DRFP.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data",  "training_results", "MSE_CV_xgboost_DRFP.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgbo

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))

np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_DRFP.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_DRFP.npy"), test_Y)

In [ ]:
save_pickel_model(bst, "xgboost_drfp.h5")

In [ ]:
y_test_pred_drfp = y_test_pred

## 4. Training a model with only reaction information (difference fingerprint):

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(list(data_train["difference_fp"]))
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["difference_fp"]))
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {'learning_rate': 0.14154883958006167,
         'max_delta_step': 0.02234358170535966,
         'max_depth': 10.869653004093198,
         'min_child_weight': 1.7936882442746056,
         'num_rounds': 361.6168542774665,
         'reg_alpha': 4.825525325323308,
         'reg_lambda': 2.74944090578774}


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_diff_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_diff_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_diff_fp.npy"), np.array(R2))
'''

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_diff_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_diff_fp.npy"), test_Y)

In [ ]:
save_pickel_model(bst, "xgboost_diff.h5")

In [ ]:
y_test_pred_diff_fp = y_test_pred

## 5. Training a model with only reaction information (structural fingerprint):

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(data_train["structural_fp"].tolist())
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["structural_fp"]))
test_Y = np.array(list(data_test["log10_kcat"]))

In [ ]:
train_X.shape, test_X.shape

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {'learning_rate': 0.01126910440903659,
         'max_delta_step': 0.5777120839605732,
         'max_depth': 5.486901609313889,
         'min_child_weight': 6.14467742389769,
         'num_rounds': 488.943459090126,
         'reg_alpha': 4.629840853377147,
         'reg_lambda': 2.1047561335691745}

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_str_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_str_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_str_fp.npy"), np.array(R2))
'''

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X)

bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_str_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_str_fp.npy"), test_Y)

In [ ]:
save_pickel_model(bst, "xgboost_structFp.h5")

## 6. Training a model with enzyme and reaction information (ESM1b_ts/DRFP):

In [ ]:
train_X = np.array(list(data_train["DRFP"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM1b_ts"]))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["DRFP"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_ts"]))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 6,14),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param =  {
    'learning_rate': 0.0685249638804391,
    'max_depth': 9,
    'reg_lambda': 3.6231955480293934,
    'reg_alpha': 3.8923304126950957,
    'max_delta_step': 1.0371427260254895,
    'min_child_weight': 14.209819958496073,
    # 'sampling_method': 'gradient_based',
    'num_rounds': 400.9598325756988,
    }


num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_DRFP.npy"), np.array(R2))
'''

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
# MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
MSE_dif_fp_test = mean_squared_error(np.reshape(test_Y, (-1)), y_test_pred)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_DRFP.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_DRFP.npy"), test_Y)
save_pickel_model(bst, "xgboost_esm1bts_drfp.h5")

### Pearson = 0.656 MSE = 0.766 R2 = 0.427

In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred

## 7. Training a model with enzyme and reaction information (ESM1b_ts/diff_fp):

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(list(data_train["difference_fp"]))
train_X = np.concatenate([train_X, np.array(list(data_train["ESM1b_ts"]))], axis = 1)
train_Y = np.array(list(data_train["log10_kcat"]))

test_X = np.array(list(data_test["difference_fp"]))
test_X = np.concatenate([test_X, np.array(list(data_test["ESM1b_ts"]))], axis = 1)
test_Y = np.array(list(data_test["log10_kcat"]))

#### (b) Hyperparameter optimization:

In [ ]:
'''def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)''';

#### (c) Training and validating model:

In [ ]:
param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))\n'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_diff_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_diff_fp.npy"), test_Y)

save_pickel_model(bst, "xgboost_esm1b_ts_diff.h5")

0.64 0.806 0.397


In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred

## Training model using all features

In [ ]:
train_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_train_indices.npy"), allow_pickle = True))
test_indices = list(np.load(join("..", "..", "data", "kcat_data", "splits", "CV_test_indices.npy"), allow_pickle = True))

In [ ]:
data_train = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "train_df_kcat_ours.pkl"))
data_test = pd.read_pickle(join("..", "..", "data", "kcat_data", "splits", "test_df_kcat_ours.pkl"))

# print((train_indices))
# print((test_indices))

data_train.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
data_test.rename(columns = {"geomean_kcat" :"log10_kcat"}, inplace = True)
len(data_train), len(data_test)

(3417, 839)

In [ ]:
data_train.columns[:-7]

Index(['Reaction ID', 'Sequence ID', 'kcat_values', 'Uniprot IDs',
       'from_BRENDA', 'from_Sabio', 'from_Uniprot', 'checked', 'Sequence',
       'substrates',
       ...
       '765', '766', '767', '771', '772', '773', '774', '775', '776', '777'],
      dtype='object', length=2270)

In [ ]:
np.array(data_train['structural_fp'].tolist())[0]

array([1, 1, 0, ..., 0, 0, 0])

#### (a) Creating input matrices:

In [ ]:
train_X = np.array(list(data_train["ESM1b"]))
print(train_X.shape)
test_X = np.array(list(data_test["ESM1b"]))
for col in data_train.columns[17:21]:
  print(col)
  if col == 'structural_fp':
    train_X = np.concatenate([train_X, np.array(data_train[col].tolist(), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(data_test[col].tolist(), dtype='float64')], axis = 1)
  else:
    train_X = np.concatenate([train_X, np.array(list(data_train[col]), dtype='float64')], axis = 1)
    test_X = np.concatenate([test_X, np.array(list(data_test[col]), dtype='float64')], axis = 1)

(3417, 1280)
ESM1b_ts
structural_fp
difference_fp
DRFP


In [ ]:
train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_norm']), dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_norm']), dtype='float64')], axis = 1)
train_X = np.concatenate([train_X, np.array(list(data_train['ESM1b_ts_norm']), dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(list(data_test['ESM1b_ts_norm']), dtype='float64')], axis = 1)

In [ ]:
train_X = np.concatenate([train_X, np.array(data_train.iloc[:,22:-7], dtype='float64')], axis = 1)
test_X = np.concatenate([test_X, np.array(data_test.iloc[:,22:-7], dtype='float64')], axis = 1)

In [ ]:
train_X = train_X.astype('float64')
test_X = test_X.astype('float64')

In [ ]:
train_Y = np.array(list(data_train["log10_kcat"]))
test_Y = np.array(list(data_test["log10_kcat"]))

In [ ]:
train_X.shape

(3417, 15560)

In [ ]:
type(train_X)

numpy.ndarray

#### (b) Hyperparameter optimization:

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index  = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)),  y_valid_pred))
    return(-np.mean(R2))


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4,12),
    #"subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds":  hp.uniform("num_rounds", 20, 200)}


trials = Trials()
best = fmin(fn = cross_validation_mse_gradient_boosting, space = space_gradient_boosting,
            algo=rand.suggest, max_evals = 200, trials=trials)

 87%|████████▋ | 174/200 [5:08:10<40:19, 93.06s/trial, best loss: -0.3735840090139679]

In [ ]:
best

#### (c) Training and validating model:

In [ ]:
param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:

R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_all.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_all.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_all.npy"), np.array(R2))


[0.6133773796060704, 0.5927733649751115, 0.5696579476183089, 0.5664901288879961, 0.6059675908688378]
[0.8995350132420895, 0.8907148311609805, 0.9570658132750683, 1.0011614220188685, 0.9055139379678919]
[0.37289620835138915, 0.3481887641919241, 0.3220370010024399, 0.31963431757814464, 0.3656110621769998]


In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_all.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_all.npy"), test_Y)

save_pickel_model(bst, "xgboost_all.h5")

0.613 0.835 0.375


In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred

## 8. Training a model with enzyme and reaction information (ESM1b_ts/DRFP/New Features):

#### (a) Creating input matrices:

In [ ]:
for col in data_train.columns:
    if "_combined" in col:
        print(col)

maccs_keys_combined
chemical_descriptors_combined
aa_composition_combined
dp_composition_combined
moreau_broto_auto_combined
moran_auto_combined
geary_auto_combined
ctd_combined
apaac_combined
socn_combined
qso_combined
traid_combined
paac_combined


### a. (ESM1b_ts/DRFP/paac)

In [ ]:
train_X = np.concatenate(
    [
        np.array(list(data_train["ESM1b_ts"])),
        np.array(list(data_train["DRFP"])),
        np.array(list(data_train["paac_combined"]))


    ], axis = 1)

test_X = np.concatenate(
    [
        np.array(list(data_test["ESM1b_ts"])),
        np.array(list(data_test["DRFP"])),
        np.array(list(data_test["paac_combined"]))


    ], axis = 1)


train_Y = np.array(list(data_train["log10_kcat"]))
test_Y = np.array(list(data_test["log10_kcat"]))

train_X.shape

(3417, 3368)

#### (b) Hyperparameter optimization:

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label=train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(
            np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred) ** 2)
        )
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    return -np.mean(R2)


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4, 12),
    # "subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds": hp.uniform("num_rounds", 20, 200),
}


trials = Trials()
best = fmin(
    fn=cross_validation_mse_gradient_boosting,
    space=space_gradient_boosting,
    algo=rand.suggest,
    max_evals=100,
    trials=trials,
)

 94%|█████████▍| 94/100 [37:21<02:28, 24.67s/trial, best loss: -0.33282023950349704]

In [ ]:
best

#### (c) Training and validating model:

In [ ]:
param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))\n'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_diff_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_diff_fp.npy"), test_Y)

save_pickel_model(bst, "xgboost_esm1b_ts_diff.h5")

0.617 0.904 0.378


In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred

### a. (ESM1b_ts/DRFP/maccs_keys)

In [ ]:
train_X = np.concatenate(
    [
        np.array(list(data_train["ESM1b_ts"])),
        np.array(list(data_train["DRFP"])),
        np.array(list(data_train["maccs_keys_combined"]))


    ], axis = 1)

test_X = np.concatenate(
    [
        np.array(list(data_test["ESM1b_ts"])),
        np.array(list(data_test["DRFP"])),
        np.array(list(data_test["maccs_keys_combined"]))


    ], axis = 1)


train_Y = np.array(list(data_train["log10_kcat"]))
test_Y = np.array(list(data_test["log10_kcat"]))

train_X.shape

(3417, 3495)

#### (b) Hyperparameter optimization:

In [ ]:
def cross_validation_mse_gradient_boosting(param):
    num_round = param["num_rounds"]
    del param["num_rounds"]
    param["max_depth"] = int(np.round(param["max_depth"]))
    param["tree_method"] = "gpu_hist"
    param["sampling_method"] = "gradient_based"

    MSE = []
    R2 = []
    for i in range(5):
        train_index, test_index = train_indices[i], test_indices[i]
        dtrain = xgb.DMatrix(train_X[train_index], label=train_Y[train_index])
        dvalid = xgb.DMatrix(train_X[test_index])
        bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)
        y_valid_pred = bst.predict(dvalid)
        MSE.append(
            np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred) ** 2)
        )
        R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    return -np.mean(R2)


from hyperopt import fmin, tpe, rand, hp, Trials

space_gradient_boosting = {
    "learning_rate": hp.uniform("learning_rate", 0.01, 1),
    "max_depth": hp.uniform("max_depth", 4, 12),
    # "subsample": hp.uniform("subsample", 0.7, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 5),
    "reg_alpha": hp.uniform("reg_alpha", 0, 5),
    "max_delta_step": hp.uniform("max_delta_step", 0, 5),
    "min_child_weight": hp.uniform("min_child_weight", 0.1, 15),
    "num_rounds": hp.uniform("num_rounds", 20, 200),
}


trials = Trials()
best = fmin(
    fn=cross_validation_mse_gradient_boosting,
    space=space_gradient_boosting,
    algo=rand.suggest,
    max_evals=200,
    trials=trials,
)

100%|██████████| 200/200 [1:12:37<00:00, 21.79s/trial, best loss: -0.33368787814371004]


In [ ]:
best

{'learning_rate': 0.056292817381962935,
 'max_delta_step': 2.1919961147483304,
 'max_depth': 8.771014052426466,
 'min_child_weight': 14.294138981688935,
 'num_rounds': 132.03790870507578,
 'reg_alpha': 1.609700793751213,
 'reg_lambda': 3.3764779548975925}

#### (c) Training and validating model:

In [ ]:
param = {
    'learning_rate': 0.15638876027725748,
    'max_depth': 8,
    'reg_lambda': 2.344616112944763,
    'reg_alpha': 4.262441473730964,
    'max_delta_step': 0.9349225200130806,
    'min_child_weight': 8.26447488212503,
    'num_rounds': 400.69265795096726,
    # 'sampling_method': 'gradient_based'
    }

num_round = param["num_rounds"]
param["max_depth"] = int(np.round(param["max_depth"]))

del param["num_rounds"]

In [ ]:
'''
R2 = []
MSE = []
Pearson = []

for i in range(5):
    train_index, test_index  = train_indices[i], test_indices[i]
    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])
    dvalid = xgb.DMatrix(train_X[test_index])

    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

    y_valid_pred = bst.predict(dvalid)
    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))
    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))
    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])

print(Pearson)
print(MSE)
print(R2)

np.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))
np.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))
np.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))
'''

'\nR2 = []\nMSE = []\nPearson = []\n\nfor i in range(5):\n    train_index, test_index  = train_indices[i], test_indices[i]\n    dtrain = xgb.DMatrix(train_X[train_index], label = train_Y[train_index])\n    dvalid = xgb.DMatrix(train_X[test_index])\n\n    bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)\n\n    y_valid_pred = bst.predict(dvalid)\n    MSE.append(np.mean(abs(np.reshape(train_Y[test_index], (-1)) - y_valid_pred)**2))\n    R2.append(r2_score(np.reshape(train_Y[test_index], (-1)), y_valid_pred))\n    Pearson.append(stats.pearsonr(np.reshape(train_Y[test_index], (-1)), y_valid_pred)[0])\n\nprint(Pearson)\nprint(MSE)\nprint(R2)\n\nnp.save(join("..", "..", "data", "training_results", "Pearson_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(Pearson))\nnp.save(join("..", "..", "data", "training_results", "MSE_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(MSE))\nnp.save(join("..", "..", "data", "training_results", "R2_CV_xgboost_ESM1b_ts_diff_fp.npy"), np.array(R2))\n'

In [ ]:
dtrain = xgb.DMatrix(train_X, label = train_Y)
dtest = xgb.DMatrix(test_X, label = test_Y)


bst = xgb.train(param, dtrain, int(num_round), verbose_eval=False)

y_test_pred = bst.predict(dtest)
MSE_dif_fp_test = np.mean(abs(np.reshape(test_Y, (-1)) - y_test_pred)**2)
R2_dif_fp_test = r2_score(np.reshape(test_Y, (-1)), y_test_pred)
Pearson = stats.pearsonr(np.reshape(test_Y, (-1)), y_test_pred)

print(np.round(Pearson[0],3) ,np.round(MSE_dif_fp_test,3), np.round(R2_dif_fp_test,3))


np.save(join("..", "..", "data", "training_results", "y_test_pred_xgboost_ESM1b_ts_diff_fp.npy"), bst.predict(dtest))
np.save(join("..", "..", "data", "training_results", "y_test_true_xgboost_ESM1b_ts_diff_fp.npy"), test_Y)

save_pickel_model(bst, "xgboost_esm1b_ts_diff.h5")

0.617 0.904 0.378


In [ ]:
y_test_pred_esm1b_ts_drfp = y_test_pred